In [2]:
import pandas as pd
from pathlib import Path

# Data Loading

In [7]:
dir = Path("~/Data")
dir = dir / "CMI-Wrist"

In [ ]:
train_data_csv = dir / "train.csv"
train_demo_csv = dir / "train_demographics.csv"
df = pd.read_csv(train_data_csv)
df.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
df_d = pd.read_csv(train_demo_csv)
df_d.head()

,subject,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SUBJ_000206,1,41,1,1,172.0,50,25.0
1,SUBJ_001430,0,11,0,1,167.0,51,27.0
2,SUBJ_002923,1,28,1,0,164.0,54,26.0
3,SUBJ_003328,1,33,1,1,171.0,52,25.0
4,SUBJ_004117,0,15,0,1,184.0,54,28.0


In [11]:
df.to_pickle(dir / "train.pkl")
df_d.to_pickle(dir / "train_demo.pkl")

In [12]:
# merging
df_m = pd.merge(df,df_d,how="left",on="subject")
df_m.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0


In [15]:
df_m.shape, df.shape

((574945, 348), (574945, 341))

In [16]:
df_m.to_pickle(dir / "train_merged.pkl")

# Data Basic Splitting
Will use holdout method first for looking at basics of the dataset. Will split this into train and val sets in the hopes of preventing data leakage. Will eventually use stratified k-folds for training.

In [ ]:
pd.read_pickle(dir / "train_merged.pkl")

In [17]:
df_y = df_m['gesture']
df_X = df_m.drop(columns=['gesture'])

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_X, df_y, stratify=df_y, random_state=123)

In [25]:
X_train.to_pickle(dir / "X_train_explore.pkl")
y_train.to_pickle(dir / "y_train_explore.pkl")

# PCA Initial Analysis

In [23]:
y_train.value_counts()

gesture
Text on phone                                 43846
Neck - scratch                                42464
Eyebrow - pull hair                           33229
Forehead - scratch                            30692
Forehead - pull hairline                      30601
Above ear - pull hair                         30420
Neck - pinch skin                             30380
Eyelash - pull hair                           30163
Cheek - pinch skin                            30093
Wave hello                                    25767
Write name in air                             23450
Pull air toward your face                     23057
Feel around in tray and pull out an object    12836
Glasses on/off                                10157
Drink from bottle/cup                          9820
Scratch knee/leg skin                          9246
Write name on leg                              7604
Pinch knee/leg skin                            7383
Name: count, dtype: int64

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
